In [1]:
!python -V

Python 3.11.9


In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


pd.set_option('display.float_format', lambda x: '%.1f' % x)

Matplotlib is building the font cache; this may take a moment.


In [3]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,1.0,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.1,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.5,1.0,N,48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.9,1.0,N,138,7,1,12.1,7.2,0.5,0.0,0.0,1.0,20.9,0.0,1.2
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.4,1.0,N,107,79,1,11.4,1.0,0.5,3.3,0.0,1.0,19.7,2.5,0.0


In [5]:
df.shape

(3066766, 19)

In [6]:
#Q1
print(f'Columns in tripdata for January 2023 is: {df.columns.size}')

Columns in tripdata for January 2023 is: 19


In [7]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds() / 60
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,1.0,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,8.4
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.1,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,6.3
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.5,1.0,N,48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0,12.8
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.9,1.0,N,138,7,1,12.1,7.2,0.5,0.0,0.0,1.0,20.9,0.0,1.2,9.6
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.4,1.0,N,107,79,1,11.4,1.0,0.5,3.3,0.0,1.0,19.7,2.5,0.0,10.8


In [8]:
#Q2
print(f"The standard deviation of the duration column is: {df['duration'].std():.2f}")

The standard deviation of the duration column is: 42.59


In [9]:
#Q3
filtered_df = df[np.logical_and((df['duration'] >= 1), (df['duration'] <= 60))]
fraction_left = len(filtered_df)/len(df) * 100
df = filtered_df.copy()
print(f"The fraction of records left after dropping outliers is: {fraction_left:.2f}")

The fraction of records left after dropping outliers is: 98.12


In [10]:
df['duration'].describe() #To verify our duration column contains only values from 1 minute to 60 minutes

count   3009173.0
mean         14.2
std           9.9
min           1.0
25%           7.2
50%          11.6
75%          18.2
max          60.0
Name: duration, dtype: float64

In [11]:
#Q4
#Change the model features to strings before One Hot encoding
features = df[['PULocationID', 'DOLocationID']]
features = features.astype(str)

In [12]:
X = features.to_dict(orient='records')
dv = DictVectorizer()
dv.fit(X)
X_train = dv.transform(X)
y_train = df['duration'].values
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

In [13]:
rmse = mean_squared_error(y_train, y_pred, squared = False)
print(f"RMSE: {rmse}")

RMSE: 7.649261821566201


#Validation dataset

In [14]:
# Predicting February trip dataset
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [15]:
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.3,1.0,N,142,163,2,4.4,3.5,0.5,0.0,0.0,1.0,9.4,2.5,0.0
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.0,1.0,N,71,71,4,-3.0,-1.0,-0.5,0.0,0.0,-1.0,-5.5,0.0,0.0
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.0,1.0,N,71,71,4,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.8,1.0,N,132,26,1,70.9,2.2,0.5,0.0,0.0,1.0,74.7,0.0,1.2
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.2,1.0,N,161,145,1,17.0,1.0,0.5,3.3,0.0,1.0,25.3,2.5,0.0


In [16]:
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds()/60
df_val = df_val[np.logical_and((df_val['duration'] >= 1), (df_val['duration'] <= 60))]
df_val.sample()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
1975919,2,2023-02-20 14:04:05,2023-02-20 14:14:45,2.0,1.9,1.0,N,161,263,1,12.1,0.0,0.5,3.2,0.0,1.0,19.3,2.5,0.0,10.7


In [17]:
df_val['duration'].describe() #To verify our duration column contains only values from 1 minute to 60 minutes

count   2855951.0
mean         14.5
std          10.1
min           1.0
25%           7.4
50%          11.8
75%          18.6
max          60.0
Name: duration, dtype: float64

In [18]:
features_val = df_val[['PULocationID', 'DOLocationID']]
features_val = features_val.astype(str)
X = features_val.to_dict(orient='records')

In [19]:
X_val = dv.transform(X)
y_pred = lr.predict(X_val)

In [20]:
y_val = df_val['duration'].values
rmse_val = mean_squared_error(y_val, y_pred, squared = False)
print(f"RMSE on validation dataset: {rmse_val}")

RMSE on validation dataset: 7.811821965108365


In [21]:
model_path = "./models/lin_reg.bin"
with open(model_path, 'wb') as f_out:
        pickle.dump((dv, lr), f_out)